Super sloppy code to conver the part indices from those used in the details-3.0.* api to that used in details-4.0.*. 

The new part indices are much more simplified, not differentiating between left and right sides of a train, merging all train car parts, etc. Plus the original dictionary had some erratic part attributes like giving the "beak" class to many of the other objects (just in the dictionary, not in the actual data, I believe)

In [87]:
import os
import json

In [13]:
def get_data_files():
    fnames = []
    cwd = os.getcwd()
    thisdir = os.path.normpath(os.path.abspath(cwd))
    for f in os.listdir(thisdir):
        ftype = os.path.splitext(f)
        if ftype[-1] != '.json':
            continue
        fnames.append(os.path.join(thisdir,f))
    return fnames

In [14]:
data_files = get_data_files()

In [18]:
_testf = data_files[0]

In [73]:
with open('analytics_cache/v32v4dict.json', 'r') as f:
    v32v4dict = json.load(f)
    v32v4dict = {int(k): {int(k2): v for k2, v in vals.items()} for k, vals in v32v4dict.items()}
    v32v4dict[113][113] = v32v4dict[113][190]

In [79]:
os.path.splitext(os.path.split(data_files[0])[-1])

('thresholded_gt_BYNAME_bird_centered', '.json')

In [81]:
for data_file in data_files:
    _testdat = []
    with open(data_file, 'r') as f:
        for l in f.readlines():
            _testdat.append(json.loads(l.strip()))  
    for HIT in _testdat:
        for ques in HIT:
            old_data = ques.values()[0]['points']['data']
            keys = list(old_data)
            ques.values()[0]['points']['data'] = {}
            for key in keys:
                # ... do stuff ...
                keydat = [int(el) for el in key.split("_")]
                keydat[-1] = v32v4dict[int(keydat[0])][int(keydat[2])]
                keydat= [str(el) for el in keydat]
                newkey = "_".join(keydat)
                ques.values()[0]['points']['data'][newkey] = old_data[key]
    dirloc, fname = os.path.split(data_file)
    _fname, ext = os.path.splitext(fname)
    _fname += "_v4"
    savename = os.path.join(dirloc, _fname+ext)
    with open(savename, 'w+') as f:
        for HIT in _testdat:
            f.write("{}\n".format(json.dumps(HIT)))